<a href="https://colab.research.google.com/github/wandb/edu/blob/main/llm-structured-extraction/2.tips.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{llmeng-1-nb2} -->

# General Tips on Prompting

Before we get into some big applications of schema engineering I want to equip you with the tools for success.
This notebook is to share some general advice on using prompts to get the most of your models.

Before you might think of prompt engineering as massaging this wall of text, almost like coding in a notepad. But with schema engineering you can get a lot more out of your prompts with a lot less work.


# Setup Colab

Run this code if you're using Google Colab, you can skip if you're running locally. You may need to restart Colab after installing requirements. 

In [1]:
from pathlib import Path

# Download files on colab
if not Path("requirements.txt").exists():
    !wget https://raw.githubusercontent.com/wandb/edu/main/llm-structured-extraction/{requirements.txt,helpers.py}
    !pip install -r requirements.txt -Uqq

In [2]:
import os
from getpass import getpass
import openai

# Setup your Openai API key
if os.getenv("OPENAI_API_KEY") is None:
  if any(['VSCODE' in x for x in os.environ.keys()]):
    print('Please enter password in the VS Code prompt at the top of your VS Code window!')
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")
  openai.api_key = os.getenv("OPENAI_API_KEY", "")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

Please enter password in the VS Code prompt at the top of your VS Code window!
OpenAI API key configured


## Using Weave for LLM Experiment Tracking

[Weave](https://wandb.github.io/weave/) is a lightweight toolkit by Weights & Biases for tracking and evaluating LLM applications. It allows you to:

- Log and debug language model inputs, outputs, and traces
- Build rigorous evaluations for LLM use cases
- Organize information across the LLM workflow

OpenAI calls are automatically logged to Weave.
`@weave.op()` allows you to log additional information to Weave.


In [3]:
import weave
weave.init("llmeng-1-nb2")

Logged in as Weights & Biases user: a-sh0ts.
View Weave data at https://wandb.ai/a-sh0ts/llmeng-1-nb2/weave


## Classification

For classification we've found there are generally two methods of modeling.

1. using Enums
2. using Literals

Use an enum in Python when you need a set of named constants that are related and you want to ensure type safety, readability, and prevent invalid values. Enums are helpful for grouping and iterating over these constants.

Use literals when you have a small, unchanging set of values that you don't need to group or iterate over, and when type safety and preventing invalid values is less of a concern. Literals are simpler and more direct for basic, one-off values.


In [4]:
import instructor
from openai import OpenAI

from enum import Enum
from pydantic import BaseModel, Field
from typing_extensions import Literal


client = instructor.patch(OpenAI())


# Tip: Do not use auto() as they cast to 1,2,3,4
class House(Enum):
    Gryffindor = "gryffindor"
    Hufflepuff = "hufflepuff"
    Ravenclaw = "ravenclaw"
    Slytherin = "slytherin"


class Character(BaseModel):
    age: int
    name: str
    house: House

    def say_hello(self):
        print(
            f"Hello, I'm {self.name}, I'm {self.age} years old and I'm from {self.house.value.title()}"
        )


resp = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[{"role": "user", "content": "Harry Potter"}],
    response_model=Character,
)
resp.model_dump()

🍩 https://wandb.ai/a-sh0ts/llmeng-1-nb2/r/call/329389c8-d00b-440b-ba16-d960bb7038bc


{'age': 17, 'name': 'Harry Potter', 'house': <House.Gryffindor: 'gryffindor'>}

In [5]:
resp.say_hello()

Hello, I'm Harry Potter, I'm 17 years old and I'm from Gryffindor


In [6]:
class Character(BaseModel):
    age: int
    name: str
    house: Literal["Gryffindor", "Hufflepuff", "Ravenclaw", "Slytherin"]


resp = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[{"role": "user", "content": "Harry Potter"}],
    response_model=Character,
)
resp.model_dump()

🍩 https://wandb.ai/a-sh0ts/llmeng-1-nb2/r/call/6f2cd8e4-ba42-40a9-a1c2-a7a53ae7ca65


{'age': 17, 'name': 'Harry Potter', 'house': 'Gryffindor'}

## Arbitrary properties

Often times there are long properties that you might want to extract from data that we can not specify in advance. We can get around this by defining an arbitrary key value store like so:


In [7]:
from typing import List


class Property(BaseModel):
    key: str = Field(description="Must be snake case")
    value: str


class Character(BaseModel):
    age: int
    name: str
    house: Literal["Gryffindor", "Hufflepuff", "Ravenclaw", "Slytherin"]
    properties: List[Property]


resp = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[{"role": "user", "content": "Snape from Harry Potter"}],
    response_model=Character,
)
resp.model_dump()

🍩 https://wandb.ai/a-sh0ts/llmeng-1-nb2/r/call/c9df45e4-52b0-49f9-b4ba-0f70743befb8


{'age': 38,
 'name': 'Severus Snape',
 'house': 'Slytherin',
 'properties': [{'key': 'gender', 'value': 'male'},
  {'key': 'position', 'value': 'Potions Master'},
  {'key': 'loyalty', 'value': 'Dumbledore'},
  {'key': 'patronus', 'value': 'doe'}]}

## Limiting the length of lists

In later chapters we'll talk about how to use validators to assert the length of lists but we can also use prompting tricks to enumerate values. Here we'll define an index to count the properties.

In the following example instead of extraction we're going to work on generation instead.


In [8]:
class Property(BaseModel):
    index: str = Field(..., description="Monotonically increasing ID")
    key: str = Field(description="Must be snake case")
    value: str


class Character(BaseModel):
    age: int
    name: str
    house: Literal["Gryffindor", "Hufflepuff", "Ravenclaw", "Slytherin"]
    properties: List[Property] = Field(
        ...,
        description="Numbered list of arbitrary extracted properties, should be exactly 5",
    )


resp = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[{"role": "user", "content": "Snape from Harry Potter"}],
    response_model=Character,
)
resp.model_dump()

🍩 https://wandb.ai/a-sh0ts/llmeng-1-nb2/r/call/723c68b1-a732-4705-b657-7b26a950b09b


InstructorRetryException: RetryError[<Future at 0x128ffbb90 state=finished raised ValidationError>]

## Defining Multiple Entities

Now that we see a single entity with many properties we can continue to nest them into many users! If we add the `Iterable` type to the `User` type we can define multiple users in a single prompt, now instead of extracting one user we can extract many users. But only after the completion of the prompt.

In [9]:
from typing import Iterable


class Character(BaseModel):
    age: int
    name: str
    house: Literal["Gryffindor", "Hufflepuff", "Ravenclaw", "Slytherin"]


resp = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[{"role": "user", "content": "Five characters from Harry Potter"}],
    response_model=Iterable[Character],
)

for character in resp:
    print(character)

🍩 https://wandb.ai/a-sh0ts/llmeng-1-nb2/r/call/ef8dd56c-8a57-4871-855e-ea6a693a2b20
age=17 name='Harry Potter' house='Gryffindor'
age=17 name='Ron Weasley' house='Gryffindor'
age=17 name='Hermione Granger' house='Gryffindor'
age=16 name='Draco Malfoy' house='Slytherin'
age=15 name='Luna Lovegood' house='Ravenclaw'


Now lets look at an example of how we can use this to generate multiple users while streaming. We can also generate tasks as the tokens are streamed in by defining an `Iterable[T]` type and setting the `stream` parameter to `True`. Now, we'll yield each user as they are generated improving the performance of our model by decreasing the time it takes to return a single result.

In [10]:
from typing import Iterable


class Character(BaseModel):
    age: int
    name: str
    house: Literal["Gryffindor", "Hufflepuff", "Ravenclaw", "Slytherin"]


resp = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[{"role": "user", "content": "Five characters from Harry Potter"}],
    stream=True,
    response_model=Iterable[Character],
)

for character in resp:
    print(character)

age=17 name='Harry Potter' house='Gryffindor'
age=17 name='Hermione Granger' house='Gryffindor'
age=17 name='Ron Weasley' house='Gryffindor'
age=16 name='Draco Malfoy' house='Slytherin'
age=16 name='Luna Lovegood' house='Ravenclaw'
🍩 https://wandb.ai/a-sh0ts/llmeng-1-nb2/r/call/e15bd6db-930f-46a2-893f-42e2b7a9869e


## Defining Relationships

Now only can we define lists of users, with list of properties one of the more interesting things I've learned about prompting is that we can also easily define lists of references.


In [11]:
class Character(BaseModel):
    id: int
    name: str
    friends_array: List[int] = Field(description="Relationships to their friends using the id")


resp = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[{"role": "user", "content": "5 kids from Harry Potter"}],
    stream=True,
    response_model=Iterable[Character],
)

for character in resp:
    print(character)

id=1 name='Harry Potter' friends_array=[2, 3, 4, 5, 6]
id=2 name='Hermione Granger' friends_array=[1, 3, 4, 6]
id=3 name='Ron Weasley' friends_array=[1, 2, 4, 6]
id=4 name='Neville Longbottom' friends_array=[1, 2, 3, 5]
id=5 name='Luna Lovegood' friends_array=[1, 4, 6]
id=6 name='Draco Malfoy' friends_array=[1, 2, 3, 5]
🍩 https://wandb.ai/a-sh0ts/llmeng-1-nb2/r/call/5cedd017-d328-41ff-8f91-945fc5ddf8d5


With the tools we've discussed, we can find numerous real-world applications in production settings. These include extracting action items from transcripts, generating fake data, filling out forms, and creating objects that correspond to generative UI. These simple tricks will be highly useful.


# Missing Data

The Maybe pattern is a concept in functional programming used for error handling. Instead of raising exceptions or returning None, you can use a Maybe type to encapsulate both the result and potential errors.

This pattern is particularly useful when making LLM calls, as providing language models with an escape hatch can effectively reduce hallucinations.

In [12]:
from typing import Optional

class Character(BaseModel):
    age: int
    name: str

class MaybeCharacter(BaseModel):
    result: Optional[Character] = Field(default=None)
    error: bool = Field(default=False)
    message: Optional[str]

In [13]:
@weave.op()
def extract(content: str) -> MaybeCharacter:
    return client.chat.completions.create(
        model="gpt-3.5-turbo",
        response_model=MaybeCharacter,
        messages=[
            {"role": "user", "content": f"Extract `{content}`"},
        ],
    )

In [14]:
extract("Harry Potter")

🍩 https://wandb.ai/a-sh0ts/llmeng-1-nb2/r/call/e5074774-054f-414d-8fc7-5dba97e396f9


InstructorRetryException: RetryError[<Future at 0x129531e50 state=finished raised ValidationError>]

In [15]:
user = extract("404 Error")

if user.error:
    raise ValueError(user.message)

🍩 https://wandb.ai/a-sh0ts/llmeng-1-nb2/r/call/97f0a2f1-2e10-4b4c-bed2-a11cbd2e18d1
